In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import pdb
import matplotlib.pyplot as plt
import scipy

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import time

import numpy as np



In [424]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [425]:
BATCHSIZE=100

transform_train = transforms.Compose([
torchvision.transforms.Grayscale(),
torchvision.transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
   root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(
   trainset, batch_size=BATCHSIZE, shuffle=True, num_workers=2) # Also check out pin_memory if using GPU
testloader = torch.utils.data.DataLoader(
    testset, batch_size=BATCHSIZE, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [427]:
total_batches = len(trainset) // BATCHSIZE * NBLEARNEPOCHS -100

print(f'Total batches: {total_batches}')

Total batches: 2400


In [55]:
total_batches = len(testset) // BATCHSIZE

print(f'Total batches: {total_batches}')

Total batches: 100


In [56]:
def mapTORCH(x, w, y):
    
    # Input shapes
    # x: (a, b, c, d) - input tensor with 'b' channels
    # w: (e, b, g, h) - weights tensor with 'b' input channels and 'e' filters
    # y: (a, e, i, j) - output tensor
    # Output of R shape: (e * g * h, b, i * j, a, 2)
    # the pairs stored are in the order: (x, y)
    # Extract the required dimensions from x, w, y
    
    start = time.time()
    batch_size, channels, _, _ = x.shape
    num_filters, _, w_height, w_width = w.shape
    _, _, out_height, out_width = y.shape
    a = time.time()
    print(f"time taken to get shapes: {a-start}")

    # Unfold x to extract sliding windows / patches
    x_unfolded = F.unfold(x, (w_height, w_width), stride=1)
    # Shape after unfolding -> (batch_size, channels * w_height * w_width, out_height * out_width)
    b = time.time()
    print(f"time taken to unfold x: {b-a}")
    # Reshape unfolded x for alignment in final tensor R
    x_unfolded = x_unfolded.transpose(1, 2).reshape(batch_size, out_height * out_width, channels, w_height * w_width)
    # x_unfolded.shape -> (batch_size, out_height * out_width, channels, w_height * w_width)
    c = time.time()
    print(f"time taken to transpose and reshape the unfolded x: {c-b}")
    
    # Reshape y for alignment
    y_permuted = y.permute(1, 2, 3, 0) # (e, i, j, a) -> (num_filters, out_height, out_width, batch_size)
    d = time.time()
    print(f"time taken to permute y: {d-c}")
    
    y_reshaped = y_permuted.reshape(num_filters, out_height * out_width, batch_size)
    # y_reshaped.shape -> (num_filters, out_height * out_width, batch_size)
    e = time.time()
    print(f"time taken to reshape permutted y: {e-d}")

    # Initialize tensor R with the new shape
    R = torch.zeros((num_filters * w_height * w_width, channels, out_height * out_width, batch_size, 2))
    f = time.time()
    print(f"time taken to init R: {f-e}")
    
    # Calculate contributions for each weight in each filter for each channel
    for n in range(num_filters):
        for j in range(w_height * w_width):
            weight_index = n * w_height * w_width + j
            R[weight_index, :, :, :, 0] = x_unfolded[:, :, :, j].permute(2, 1, 0)  # Align x patches
            R[weight_index, :, :, :, 1] = y_reshaped[n, :, :].unsqueeze(0).repeat(channels, 1, 1)  # Expand and align y
            
    g = time.time()
    print(f"time taken to fill R with the for loop: {g-f}")

    print(f"Total time: {g-start}")
    
    return R

In [57]:
import torch
import torch.nn.functional as F

def mapTORCH_optimized(x, w, y):
    batch_size, channels, _, _ = x.shape
    num_filters, _, w_height, w_width = w.shape
    _, _, out_height, out_width = y.shape

    # Unfold x to extract sliding windows / patches
    x_unfolded = F.unfold(x, (w_height, w_width), stride=1)
    x_unfolded = x_unfolded.permute(0, 2, 1).reshape(batch_size, out_height * out_width, channels, w_height, w_width)
    x_unfolded = x_unfolded.reshape(batch_size, out_height * out_width, channels, w_height * w_width)

    # Permute and reshape y for alignment
    y_permuted = y.permute(0, 2, 3, 1).reshape(batch_size, out_height * out_width, num_filters).permute(0, 2, 1)

    # Initialize tensor R
    R = torch.zeros(num_filters * w_height * w_width, channels, out_height * out_width, batch_size, 2)

    # Align x_unfolded to fit R
    x_reshaped = x_unfolded.repeat(1, 1, num_filters, 1).reshape(batch_size, out_height * out_width, num_filters * w_height * w_width, channels)
    x_reshaped = x_reshaped.permute(2, 3, 1, 0)

    # Align y_reshaped to fit R
    y_repeated = y_permuted.unsqueeze(3).repeat(1, 1, 1, channels)
    y_repeated = y_repeated.permute(2, 3, 1, 0).repeat(num_filters * w_height * w_width // (num_filters * out_height * out_width), 1, 1, 1)

    # Assign to R
    R[:, :, :, :, 0] = x_reshaped
    R[:, :, :, :, 1] = y_repeated

    return R


In [58]:
# 5 min to do the loop :)

In [32]:
# Example usage:
x = torch.rand(100, 100, 14, 14)  # Input images
w = torch.rand(196, 100, 3, 3)  # Weights
y = torch.rand(100, 196, 12, 12)  # Output

r = mapTORCH(x, w, y)


time taken to get shapes: 1.0013580322265625e-05
time taken to unfold x: 0.06861090660095215
time taken to transpose and reshape the unfolded x: 0.00018167495727539062
time taken to permute y: 2.4080276489257812e-05
time taken to reshape permutted y: 2.2411346435546875e-05
time taken to init R: 0.9681034088134766
time taken to fill R with the for loop: 259.5308156013489
Total time: 260.5677680969238


In [59]:
# Example usage:
x = torch.rand(100, 100, 14, 14)  # Input images
w = torch.rand(196, 100, 3, 3)  # Weights
y = torch.rand(100, 196, 12, 12)  # Output

rm = mapTORCH_optimized(x, w, y)


RuntimeError: The expanded size of the tensor (144) must match the existing size (196) at non-singleton dimension 2.  Target sizes: [1764, 100, 144, 100].  Tensor sizes: [0, 100, 196, 100]

In [ ]:
rm.shape

In [45]:
r.shape

torch.Size([1764, 100, 144, 100, 2])

In [46]:
rm[0,0,0,0]

tensor([0.2164, 0.9202])

In [47]:
r[0,0,0,0]

tensor([0.0529, 0.3382])

In [48]:
torch.equal(r, rm)

False

In [48]:
    def compute_pearson_coefficient_vectorized(R):
        # Space complexity: O(5×e×g×h×b×i×j×a)
        # crashed my laptop for the second layer.
        # Extract components
        x = R[..., 0]
        y = R[..., 1]
    
        # Mean across the batch dimension
        mean_x = x.mean(dim=3, keepdim=True)
        mean_y = y.mean(dim=3, keepdim=True)
    
        # Deviations from mean
        xm = x - mean_x
        ym = y - mean_y
    
        # Covariance and variance
        cov_xy = (xm * ym).mean(dim=3)
        var_x = (xm ** 2).mean(dim=3)
        var_y = (ym ** 2).mean(dim=3)
    
        # Pearson correlation
        std_x = torch.sqrt(var_x)
        std_y = torch.sqrt(var_y)
        correlation = cov_xy / (std_x * std_y)
    
        # Handle NaN and infinities possibly caused by division by zero
        correlation = torch.nan_to_num(correlation, nan=0.0)
    
        return correlation


In [420]:
NL=1                   # Number of layers 
STRIDES=(1,1,1)         # Input strides (for each layer)    
POOLSTRIDES = (2, 2, 2) # Pooling strides
POOLDIAMS = (2, 2, 2)   # Pooling diameters
SIZES = (5, 3, 3)       # Receptive field sizes
N = [100, 196, 400]     # N = number of cells per column (i.e. channels) for each layer
K = [1, 1, 1]           # K = number of winners in the winner-take-all (WTA) competition

TARGETRATE = [float(K[ii] / N[ii]) for ii in range(NL)]   #  The target winning rate of each cell *must* be K/N to allow for equilibrium

CSIZE = 32              # Input image size
FSIZE = 12              # Size of the Difference-of-Gaussians filter
NBLEARNEPOCHS = 5       # Number of epochs for Hebbian learning (2 more epochs are added for data acquisition for training and testing the linear classifier)
LR = 0.01 / BATCHSIZE   # Learning rate
MUTHRES =   10.0        # Threshold adaptation rate

USETRIANGLE = False     # Should we use Coates' "triangle" method to compute actual neural responses, or should we just use the WTA result as is?

tic = time.time()

# Initializations
w=[]
optimizers=[]
thres=[]
for numl in range(NL):
    NIC = 1 if numl == 0 else N[numl-1]  # Number of input channels for the weight filters at each layer
    wi = torch.randn((N[numl], NIC, SIZES[numl], SIZES[numl]), requires_grad=True, device=device) 
    wi.data = wi.data  / (1e-10 + torch.sqrt(torch.sum(wi.data ** 2, dim=[1,2,3], keepdim=True))) # Weights have norm 1
    w.append(wi)
    thres.append(torch.zeros((1,N[numl],1,1), requires_grad=False).to(device))  # Thresholds (i.e. adaptive biases to ensure a roughly equal firing rate)
    optimizers.append(optim.SGD((w[numl],), lr=LR, momentum=0.0))       # We use one optimizer per layer, though this is not strictly necessary with judicious use of .detach()              


# Build a difference-of-Gaussians kernel to spatially decorrelate (approximately whiten) the images
gk1 = np.zeros((FSIZE, FSIZE)); gk1[FSIZE//2, FSIZE//2] = 1
gk2 = (scipy.ndimage.gaussian_filter(gk1,sigma=.5) - scipy.ndimage.gaussian_filter(gk1,sigma=1.0))
dog = torch.Tensor(gk2[np.newaxis,np.newaxis,:,:]).to(device) #  Adding two singleton dimensions for input and output channels (1 each)



print("Initialization time:", time.time()-tic, "Device:", device)
testtargets = traintargets = []; testouts = trainouts = []
tic = time.time()
firstpass=True
nbbatches = 0


Initialization time: 0.07478499412536621 Device: cuda:0


In [50]:
# Repeat on MNIST / CF10}
# HyperParam tunning on the corr cutoff / batchsize / learning rate.
# layer tunning.

In [51]:
import time

total_function_time = 0.0

In [52]:
print_grad= False
start_time_loop = time.time()

for epoch in range(NBLEARNEPOCHS+2):

    current = time.time()
    elapsed = current - start_time_loop
    print(f"Epoch {epoch}, time elapsed: {elapsed}")

    myloader = testloader if epoch == NBLEARNEPOCHS +1 else trainloader
    for numbatch, (x, targets) in enumerate(myloader):
        
        current = time.time()
        elapsed = current - start_time_loop
        print(f"Batch {numbatch}, time elapsed: {elapsed}")

        nbbatches += 1

        # Prepare the input images and decorrelate them with the difference-of-gaussian filters
        with torch.no_grad():         
            x = x.to(device); targets = targets.to(device)
            x = x - torch.mean(x, dim=(1, 2,3),keepdim=True)
            x = x / (1e-10 + torch.std(x, dim=(1, 2,3), keepdim=True)) 
            x = F.conv2d(x, dog, groups=1, padding=FSIZE//2)    # DoG filtering 


        # Now run the network's layers in succession
        
        for numl in range(NL):

            optimizers[numl].zero_grad()  # We use one optimizer per layer for added clarity, but this is not strictly necessary (with judicious use of .detach() to stop gradient flow)

            # We normalize the input
            with torch.no_grad():
                x = x - torch.mean(x, dim=(1, 2,3),keepdim=True);  x = x / (1e-10 + torch.std(x, dim=(1, 2,3), keepdim=True)) 

            # We apply the weight convolutions, giving us the feedforward input into each cell (and building the first part of the computational graph)
            
            prelimy = F.conv2d(x, w[numl], stride=STRIDES[numl])  
            
            # Then we compute the "real" output (y) of each cell, with winner-take-all competition
            with torch.no_grad():                
                realy = (prelimy - thres[numl])
                tk = torch.topk(realy.data, K[numl], dim=1, largest=True)[0]
                realy.data[realy.data < tk.data[:,-1,:,:][:, None, :, :]] = 0       
                realy.data = (realy.data > 0).float()

            # Then we compute the surrogate output yforgrad, whose gradient computations produce the desired Hebbian output
            # Note: We must not include thresholds here, as this would not produce the expected gradient expressions. The actual values will come from realy, which does include thresholding.

            yforgrad = prelimy - 1/2 * torch.sum(w[numl] * w[numl], dim=(1,2,3))[None,:, None, None]                # Instar rule, dw ~= y(x-w)  
            # yforgrad = prelimy - 1/2 * torch.sum(w[numl] * w[numl], dim=(1,2,3))[None,:, None, None] * realy.data # Oja's rule, dw ~= y(x-yw)
            # yforgrad = prelimy                                                                                      # Plain Hebb, dw ~= xy

            yforgrad.data = realy.data # We force the value of yforgrad to be the "correct" y

            # We perform the backward pass and the learning step, which applies the desired Hebbian updates
            loss = torch.sum( -1/2 * yforgrad * yforgrad) 
            loss.backward()

            if(firstpass):
                # Just to check shapes are correct
                with torch.no_grad():
                    #print(f"----- Pre optimizer step w grad. {w[numl].grad.shape}")
                    onefilter= torch.ones_like(w[numl])
                    #print(f"--- shape w: {w[numl].shape} shape x {x.shape} shape onefiter {onefilter.shape}")

                    manual=  F.conv2d(x,w[numl],stride= 1) * F.conv2d(x,onefilter,stride=1)
                    print(f"--- Manual shape: {manual.shape} vs computed {w[numl].grad.shape}")
                    if(print_grad):
                        print(f"--- Manual content: {manual.shape} vs computed {w[numl].grad.shape}")
                        print_grad= False

            if nbbatches > 100 and epoch < NBLEARNEPOCHS:  # No weight modifications before batch 100 (burn-in) or after learning epochs (during data accumulation for training / testing)
                # Modify gradients using the coefficient vector only for the first layer
                if(False): #numl==0):
                    start_time = time.time()
                    with torch.no_grad():  # Ensure no tracking history 
                        #print(f"\n\n//////////////////// LAYER 0 starting R comp for x {x.shape}")
                        R = mapTORCH(x.clone().detach(), w[numl].clone().detach(), prelimy.clone().detach())
                        #print("done with R comp")
                        Rpearson = compute_pearson_coefficient_vectorized(R)
                        #print(f"Shape of pearson function ouput{Rpearson.shape}")
                        coeffs = Rpearson.mean(dim=2)
                        #print(f"coefficients shape : {coeffs.shape}")
                        #print(f"W grad param update shape : {w[numl].grad.shape}")
                        #print(f"//////////////////// end of Pearson comutation\n\n")
                        #print(w[numl].grad.shape)
                        coeffs = coeffs.view(w[numl].grad.shape)
                        coeffs = coeffs.to(w[numl].grad.device)  
                        w[numl].grad *= coeffs
                        #Apply coefficients
                        #print(f"W grad param update shape : {w[numl].grad.shape}")
                        
                    end_time = time.time()
                    
                    duration = end_time - start_time
                    total_function_time += duration

                optimizers[numl].step()              
                w[numl].data =  w[numl].data / (1e-10 + torch.sqrt(torch.sum(w[numl].data ** 2, dim=[1,2,3], keepdim=True)))  # Weight normalization

            # We show the sizes of the layers. Watch especially the size of the last layer - if it's just 1x1, there won't be a lot of information there.
            #if firstpass:
                #print("Layer", numl, ": x.shape:", x.shape, "y.shape (before MaxP):", realy.shape, end=" ")

            # Apply pooling to produce the input of the next layer (or the final network output)
            with torch.no_grad():
                x = F.avg_pool2d(realy, POOLDIAMS[numl], stride=POOLSTRIDES[numl]).detach()       

                #if firstpass:
                    #print("y.shape (after MaxP):", x.shape)  # The layer's final output ("y") is now x for the next step

                # Threshold adaptation is based on realy, i.e. the one used for plasticity. Always binarized (firing vs. not firing).
                thres[numl] +=  MUTHRES *   (torch.mean((realy.data > 0).float(), dim=(0,2,3))[None, :, None, None] -  TARGETRATE[numl])


        # After all layers are done

        if epoch >= NBLEARNEPOCHS:  # Collecting data to train and test a linear classifier, based on (frozen) network response to the training and testing datasets, respectively.
            # We simply collect the outputs of the network, as well as the labels. The actual training/testing will occur below, with a linear classifier.
            if epoch == NBLEARNEPOCHS:
                testtargets.append(targets.data.cpu().numpy())
                testouts.append(x.data.cpu().numpy())
            elif epoch ==NBLEARNEPOCHS + 1:
                traintargets.append(targets.data.cpu().numpy())
                trainouts.append(x.data.cpu().numpy())
            else:
                raise ValueError("Too many epochs!")

        firstpass = False 


    # After all batches for this epoch are done

    #if nbbatches % 1000 == 0: 
    print("Number of batches after epoch", epoch,  ":", nbbatches, "- time :", (time.time()-tic), "s")
    tic =  time.time()



print("Training done..")

print(f'Total time spent in Pearson Correlation: {total_function_time:.2f} seconds')




Epoch 0, time elapsed: 0.0005233287811279297
Batch 0, time elapsed: 0.9234602451324463
--- Manual shape: torch.Size([100, 100, 29, 29]) vs computed torch.Size([100, 1, 5, 5])
Batch 1, time elapsed: 1.3227262496948242
Batch 2, time elapsed: 1.3271796703338623
Batch 3, time elapsed: 1.3317968845367432
Batch 4, time elapsed: 1.338634729385376
Batch 5, time elapsed: 1.3431692123413086
Batch 6, time elapsed: 1.3512604236602783
Batch 7, time elapsed: 1.3595335483551025
Batch 8, time elapsed: 1.3712859153747559
Batch 9, time elapsed: 1.3806638717651367
Batch 10, time elapsed: 1.3869612216949463
Batch 11, time elapsed: 1.400209903717041
Batch 12, time elapsed: 1.4111595153808594
Batch 13, time elapsed: 1.4229989051818848
Batch 14, time elapsed: 1.4276244640350342
Batch 15, time elapsed: 1.4316084384918213
Batch 16, time elapsed: 1.435598611831665
Batch 17, time elapsed: 1.4396107196807861
Batch 18, time elapsed: 1.4503214359283447
Batch 19, time elapsed: 1.4544603824615479
Batch 20, time elaps

In [53]:
results = {
        'testtargets': testtargets,
        'testouts': testouts,
        'traintargets': traintargets,
        'trainouts': trainouts
    }

output_dir = './results/PearsonBatch04_05/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, 'Baseline1Lay.pth')
torch.save(results, file_path)

In [54]:
file = 'allBatches'

output_dir = './results/PearsonBatch04_05/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, 'allBatches.pth')

loaded_data = torch.load(file_path)



testtargets = loaded_data['testtargets']
testouts = loaded_data['testouts']
traintargets = loaded_data['traintargets']
trainouts = loaded_data['trainouts']


In [56]:
print(len(testtargets))

600


In [55]:

# Converting target indices to one-hot:
tgt_idx = np.hstack(traintargets)
tgt = np.zeros((tgt_idx.shape[0], 10))
tgt[np.arange(tgt_idx.shape[0]), tgt_idx] = 1
print(tgt[:4,:])
print(tgt_idx[:4])

USESVC = False  # Support Vector Classifier is slightly better, but extremely slow.

# Flattening the network outputs
zetrainouts = trainouts; zetestouts = testouts

inputs_f = [np.reshape(z, (z.shape[0], -1)) for z in zetrainouts] 
inputs = np.vstack(inputs_f)

print(tgt.shape)
print(inputs.shape)


import sklearn
from sklearn import svm
from sklearn import linear_model

tic = time.time()

if USESVC:
    clf = svm.LinearSVC(dual=False)
    clf.fit(inputs, tgt_idx)
else:
    clf = linear_model.Ridge()   # Simple linear ridge regression of the 1-hot labels over the network outputs
    clf.fit(inputs, tgt)

print("Time:", time.time() - tic)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
[8 9 2 5]
(60000, 10)
(60000, 1600)
Time: 4.315458536148071


In [5]:
tgt_idx = np.hstack(testtargets)
tgt = np.zeros((tgt_idx.shape[0], 10))
tgt[np.arange(tgt_idx.shape[0]), tgt_idx] = 1

inputs_f = [np.reshape(z, (z.shape[0], -1)) for z in zetestouts] 
inputs = np.vstack(inputs_f)

if USESVC: 
    t = clf.predict(inputs)
    print(np.mean(tgt_idx==t))

else:
    t = clf.predict(inputs)
    tm = np.argmax(t, axis=1)
    # print(tm[:10])
    # print(tgt_idx[:10])
    tacc = np.mean(tgt_idx==tm)
    print("Test accuracy:", tacc)


Test accuracy: 0.43285


In [ ]:
# do for Baseline1Lay.pth

In [158]:
x = torch.randn(2, 2, 3, 3).to(device)
w = torch.randn(3, 2, 2, 2).to(device)
y = torch.conv2d(x, w).to(device) # torch.Size([2, 3, 2, 2])

In [173]:
x[0,:,:,:]

tensor([[[-1.9840, -1.0597,  0.1119],
         [ 0.3258, -0.4912,  0.4927],
         [-1.5796,  0.0109, -0.7910]],

        [[ 0.4043, -1.7838, -0.8051],
         [ 0.5031, -0.8504, -1.1229],
         [-1.0210,  1.8296,  1.0084]]], device='cuda:0')

In [181]:
y[0, 0, :, :]

tensor([[ 2.6527,  3.4692],
        [ 3.9036, -2.3409]], device='cuda:0')

In [175]:
# Testeando Filtro 0 , Canal 1, Weight 4 (1,1) update

In [178]:
a = w[0, :, :, :] 
a=a[None, :]
a # PARA : W0 =-2.3987

tensor([[[[-2.3987, -0.6609],
          [-1.3566,  0.1799]],

         [[ 0.0257,  0.5585],
          [-1.6148,  0.5623]]]], device='cuda:0')

In [179]:
b = x[0, :, :, :]
b= b[None, :]
b #[-1.9840, -1.0597,
   #[ 0.3258, -0.4912,

tensor([[[[-1.9840, -1.0597,  0.1119],
          [ 0.3258, -0.4912,  0.4927],
          [-1.5796,  0.0109, -0.7910]],

         [[ 0.4043, -1.7838, -0.8051],
          [ 0.5031, -0.8504, -1.1229],
          [-1.0210,  1.8296,  1.0084]]]], device='cuda:0')

In [180]:
F.conv2d(b, a  ,stride=1)
# WE SHOULD GET Y = 4.9291

tensor([[[[ 2.6527,  3.4692],
          [ 3.9036, -2.3409]]]], device='cuda:0')

In [160]:
storex[0,0]

tensor([-1.9840, -1.0597,  0.3258, -0.4912], device='cuda:0')

In [172]:
# para w0: OKOKKOKKOKKO
storey[0] #channel 0

tensor(2.6527, device='cuda:0')

In [196]:
#Let's get the list and compute the correlation manually:
x1 = x[:, 0, 0:2, 0:2].reshape(x.shape[0],1, -1)
x1=x1.squeeze(1)
x1

tensor([[-1.9840, -1.0597,  0.3258, -0.4912],
        [ 0.4954, -0.3997, -0.1993, -0.7205]], device='cuda:0')

In [241]:
y1 = y[:, 0, 0, 0]
y1

tensor([ 2.6527, -1.5745], device='cuda:0')

In [242]:
x1.shape, y1.shape

(torch.Size([2, 4]), torch.Size([2]))

In [243]:
y2= y1.unsqueeze(1)
y2.shape

torch.Size([2, 1])

In [246]:
y1= y2.expand(2, 4)
y1.shape

torch.Size([2, 4])

In [254]:
def compute_correlation(x, y):
    mean_x = torch.mean(x)
    print(mean_x)
    mean_y = torch.mean(y)
    print("mean y", mean_y)
    xm = x - mean_x
    print(xm)
    print("y", y)
    ym = y - mean_y
    print("ym", ym)
    r_num = torch.sum(xm * ym)
    print(r_num)
    r_den = torch.sqrt(torch.sum(xm**2) * torch.sum(ym**2))
    print(r_den)
    r = r_num / r_den
    
    return r if r_den != 0 else 0


In [258]:
result = torch.zeros(4)
for i in range(4):
    print(f"between {x1[:, i]} and  { y1[:, i]}")
    result[i] = compute_correlation(x1[:, i], y1[:, i])

print("Correlation results:", result)

between tensor([-1.9840,  0.4954], device='cuda:0') and  tensor([ 2.6527, -1.5745], device='cuda:0')
tensor(-0.7443, device='cuda:0')
mean y tensor(0.5391, device='cuda:0')
tensor([-1.2397,  1.2397], device='cuda:0')
y tensor([ 2.6527, -1.5745], device='cuda:0')
ym tensor([ 2.1136, -2.1136], device='cuda:0')
tensor(-5.2404, device='cuda:0')
tensor(5.2404, device='cuda:0')
between tensor([-1.0597, -0.3997], device='cuda:0') and  tensor([ 2.6527, -1.5745], device='cuda:0')
tensor(-0.7297, device='cuda:0')
mean y tensor(0.5391, device='cuda:0')
tensor([-0.3300,  0.3300], device='cuda:0')
y tensor([ 2.6527, -1.5745], device='cuda:0')
ym tensor([ 2.1136, -2.1136], device='cuda:0')
tensor(-1.3949, device='cuda:0')
tensor(1.3949, device='cuda:0')
between tensor([ 0.3258, -0.1993], device='cuda:0') and  tensor([ 2.6527, -1.5745], device='cuda:0')
tensor(0.0632, device='cuda:0')
mean y tensor(0.5391, device='cuda:0')
tensor([ 0.2625, -0.2625], device='cuda:0')
y tensor([ 2.6527, -1.5745], devic

In [259]:
result

tensor([-1.0000, -1.0000,  1.0000,  1.0000])

In [261]:
coeffs

tensor([[-0.2500, -0.2500],
        [ 0.5000,  0.5000],
        [ 0.5000,  0.5000],
        [ 0.2500,  0.2500],
        [-0.2500, -0.2500],
        [-0.5000, -0.5000],
        [-0.5000, -0.5000],
        [ 0.2500,  0.2500],
        [ 0.2500,  0.2500],
        [-0.5000, -0.5000],
        [-0.5000, -0.5000],
        [-0.2500, -0.2500]], device='cuda:0')

In [412]:
x = torch.randn(100, 196, 6, 6).to(device)
w = torch.randn(400, 196, 3, 3).to(device)
y = torch.conv2d(x, w).to(device) # torch.Size([2, 3, 2, 2])
y.shape
# time taken for this one : 0.7 s -> 0.81 -> 0.78

torch.Size([100, 400, 4, 4])

In [410]:
x = torch.randn(100, 1, 33, 33).to(device)
w = torch.randn(100, 1, 5, 5).to(device)
y = torch.conv2d(x, w).to(device) # torch.Size([2, 3, 2, 2])
y.shape
# time taken for this one : 0.54 s -> 0.56

torch.Size([100, 100, 29, 29])

In [422]:
x = torch.randn(100, 100, 14, 14).to(device)
w = torch.randn(196, 100, 3, 3).to(device)
y = torch.conv2d(x, w).to(device) # torch.Size([2, 3, 2, 2])
y.shape
# time taken for this one : 0.39 s / after modification to get matrix : 0.41

torch.Size([100, 196, 12, 12])

In [367]:
x = torch.randn(2, 2, 3, 3).to(device)
w = torch.randn(3, 2, 2, 2).to(device)
y = torch.conv2d(x, w).to(device) # torch.Size([2, 3, 2, 2])
y.shape
# time taken for this one : 0.39 s

torch.Size([2, 3, 2, 2])

In [423]:
# trying parallelization for all channels:

t1 = time.time()
numf, fsize =  w.shape[0], w.shape[2]
convsize = y.shape[2]
ch = x.shape[1]

coeffsMatrix= torch.zeros_like(w, device= x.device)
print(coeffsMatrix.shape)

# there must be a way to keep this whole thing into a matrix structure.

for i in range(numf*fsize*fsize):

    # --------------------------------------------------------------------
    # Computing the current weight's alpha, beta, gamma, delta
    index = i%(fsize*fsize)
    # filter number
    alpha = int((i - (index)) /(fsize*fsize) )
    # 1 channel so far
    beta = int(ch)
    #filter weight position:
    delta = int(index% fsize)
    gamma = int((index - delta) /fsize)
    # --------------------------------------------------------------------
    
    p1 = x[:, :, gamma: gamma+convsize, delta: delta+convsize]  # x (100, 3, 33, 33) ; p1 shape (100, 3, selection,selection)
    p2 = p1.reshape(p1.shape[1],p1.shape[0], -1)  # canales 3 columnas 100 canales de 841 valores shape (3, 100, 841)

    q = y[:, alpha, gamma, delta] #100 valores shape(3, 100)
    
    # --------------------------------------------------------------------

    # Mean across the batch dimension
    mean_x = p2.mean(dim=1, keepdim=True) # ([3, 1, 841])
    mean_y = q.mean(dim=0, keepdim=True) # ([1])

    # Deviations from mean
    xm = p2 - mean_x # ([3, 100, 841]) #diff per channels
    ym = q - mean_y # ([100])

    # Covariance and variance 
    cov_xy = (xm * ym.view(1,-1, 1).expand(xm.shape[0],-1,1)).mean(dim=1) #([3,100, 841]).mean = shape ([3, 841]) # FIXED
    var_x = (xm ** 2).mean(dim=1) #shape ([3, 841])
    var_y = (ym ** 2).mean(dim=0) #shape ([1])

    # Pearson correlation
    std_x = torch.sqrt(var_x)
    std_y = torch.sqrt(var_y)

    correlation = cov_xy / (std_x * std_y)
  
    # Handle NaN and infinities possibly caused by division by zero
    correlation = torch.nan_to_num(correlation, nan=0.0)
    coeffsMatrix[alpha,:beta,gamma,delta]= correlation.mean(dim=1)[:beta]

    t2 = time.time()
    
print(f"time taken: {t2-t1}")

torch.Size([196, 100, 3, 3])
time taken: 0.44226670265197754


In [404]:
coeffs.shape

torch.Size([12, 2])

In [416]:
coeffsMatrix.shape

torch.Size([400, 196, 3, 3])

In [418]:
torch.equal(savedmat, coeffsMatrix)

True

In [414]:
savedmat = coeffsMatrix

In [444]:
pi[0][0].mean()

tensor(0.3569)

In [445]:
pi[0][1].mean()

tensor(-0.0175)

In [448]:
pi = torch.randn(100, 3, 5, 5)
#we want the average update across channels and filter : (100,3)
po=pi.reshape(*pi.shape[:2], -1)
print(po.shape)
pu = po.mean(dim= 2)

torch.Size([100, 3, 25])


In [447]:
pu.shape

torch.Size([100, 3])